# Final Project

Import important modules

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor


Read in the data

In [70]:
#urlsalary = 'https://www.basketball-reference.com/contracts/players.html'
#salary = pd.read_html(urlsalary)

urlsalary = 'https://hoopshype.com/salaries/players/2019-2020/'
salary = pd.read_html(urlsalary)

urlplayer = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html'
players = pd.read_html(urlplayer)

Clean up data for players dataset

In [83]:
players2 = players[0]
players2.head()
players2 = players2.loc[players2.Rk != 'Rk']
len(players2)

#there are some duplicate players that have switched teams. I decided to drop them because they could be outliers
#I first found all of the duplicate names and then filtered them through and dropped any rows with duplicates
result = players2.groupby('Player').count()
dupes = result.index[result['Rk']>1].tolist()
players_filtered = players2[~players2.Player.isin(dupes)]

len(players_filtered)



469

Clean up data for salary dataset

In [84]:
#change the list (as read in) into a dataframe
salary2 = salary[0]

#drop the columns that are not needed
salary2 = salary2[['Player', '2019/20']]
salary2.head()
#len(salary2)

,Player,2019/20
0,Stephen Curry,"$40,231,758"
1,Russell Westbrook,"$38,506,482"
2,Chris Paul,"$38,506,482"
3,James Harden,"$38,199,000"
4,John Wall,"$38,199,000"


Combine the two dataframes, merging on Player name

In [95]:
combined = pd.merge(salary2,players_filtered, on = 'Player')
combined.head()
#len(combined)


,Player,2019/20,Rk,Pos,Age,Tm,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Stephen Curry,"$40,231,758",121,PG,31,GSW,5,5,27.8,6.6,...,1.000,0.8,4.4,5.2,6.6,1.0,0.4,3.2,2.2,20.8
1,Russell Westbrook,"$38,506,482",506,PG,31,HOU,57,57,35.9,10.6,...,.763,1.8,6.2,7.9,7.0,1.6,0.4,4.5,3.5,27.2
2,Chris Paul,"$38,506,482",396,PG,34,OKC,70,70,31.5,6.2,...,.907,0.4,4.6,5.0,6.7,1.6,0.2,2.3,2.3,17.6
3,James Harden,"$38,199,000",198,SG,30,HOU,68,68,36.5,9.9,...,.865,1.0,5.5,6.6,7.5,1.8,0.9,4.5,3.3,34.3
4,LeBron James,"$37,436,858",250,PG,35,LAL,67,67,34.6,9.6,...,.693,1.0,6.9,7.8,10.2,1.2,0.5,3.9,1.8,25.3


Impute the means for missing numerical values

In [99]:
combined.fillna(combined.mean(),inplace=True)
combined.dropna(axis = 0, subset = ['2019/20'], inplace=True)
combined = combined.select_dtypes(exclude=['object'])

imputer = SimpleImputer(missing_values = np.NaN, strategy = 'mean')
imputer = imputer.fit(combined)
combined_imputed = pd.DataFrame(imputer.transform(combined))
combined_imputed.columns = combined.columns
combined = combined_imputed

KeyError: ['2019/20']